In [5]:
#import needed libraries
import requests
import tweepy
import pandas as pd
import os

## Gathering data

In [3]:
# download the tweet image predictions tsv file
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(os.path.join(url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [ ]:
# 
data = tweepy.